In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import torch
from torch import autograd

In [ ]:
from torch.utils.data import TensorDataset, DataLoader

In [ ]:
train = pd.read_csv('/kaggle/input/boston-house-prices/housing.csv', sep='\s+', header = None, names = ['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT', 'PRICE'])
train

In [ ]:
import sklearn
from sklearn import preprocessing

In [ ]:
names = train.columns
names

In [ ]:
scaler = preprocessing.StandardScaler()

In [ ]:
scaled_df = scaler.fit_transform(train)
scaled_df = pd.DataFrame(scaled_df, columns=names)

In [ ]:
type(scaled_df)

In [ ]:
train_tensor = torch.tensor(scaled_df.values, dtype = torch.float32)

In [ ]:
train_tensor.shape

In [ ]:
train_features = train_tensor[:,:13]

In [ ]:
train_features.shape

In [ ]:
train_labels = train_tensor[:, 13]
train_labels.shape

In [ ]:
train_labels[0]

In [ ]:
batch_size = 15
dataset = TensorDataset(train_features, train_labels)
data_iter = DataLoader(dataset, batch_size, shuffle = True)

for X, y in data_iter:
    print(X, y)
    print(X.dtype)
    print(X.shape)
    print(y.shape)
    break

In [ ]:
model = torch.nn.Sequential(torch.nn.Linear(13, 1))

In [ ]:
loss = torch.nn.MSELoss(reduction='mean')

In [ ]:
trainer = torch.optim.SGD(model.parameters(), lr = 0.001)

In [ ]:
num_epochs = 100
for epoch in range(1, num_epochs + 1):
    for X, y in data_iter:
        trainer.zero_grad()
        l = loss(model.forward(X).reshape(-1), y)
        l.backward()
        trainer.step()
    l = loss(model.forward(train_features).reshape(-1), train_labels)
    if epoch % 5 == 0:
        print('epoch %d, loss: %f' % (epoch, l.item()) , '|\tw', model[0].weight.data, '|\tb', model[0].bias.data)

In [ ]:
n = 1
y_hat = model.forward(train_features[n])
print("y_hat: ", y_hat)
print("y: ", train_labels[n])
print("Error: ", y_hat.item() - train_labels[n])

**MNIST**

In [ ]:
import torchvision
import time

In [ ]:
def transform(data):
    data = torch.tensor(data, dtype = torch.float32)
    return torch.floor(data / 128.).squeeze(axis = -1)

In [ ]:
mnist_train = pd.read_csv('/kaggle/input/digit-recognizer/train.csv')
mnist_test = pd.read_csv('/kaggle/input/digit-recognizer/test.csv')

In [ ]:
mnist_train

In [ ]:
train_lbls = mnist_train['label']
train_lbls

In [ ]:
type(train_lbls)

In [ ]:
train_features = mnist_train.drop('label', axis = 1)

In [ ]:
train_features

In [ ]:
train_tensor = transform(train_features.values)
train_tensor

In [ ]:
train_tensor[0].shape

In [ ]:
lbls_tensor = torch.tensor(train_lbls.values, dtype = torch.float32)
lbls_tensor

In [ ]:
batch_size = 256
dataset_mnist = TensorDataset(train_tensor, lbls_tensor)
train_iter = DataLoader(dataset_mnist, batch_size, shuffle = True)
test_iter = torch.utils.data.DataLoader(transform(mnist_test.values), 256)

In [ ]:
model = torch.nn.Sequential(torch.nn.Linear(784,10))
loss = torch.nn.CrossEntropyLoss()
trainer2 = torch.optim.SGD(model.parameters(), lr = 0.001)

In [ ]:
num_epochs = 10
for epoch in range(0, num_epochs):
    start, train_loss, train_acc, train_n, test_loss, test_acc, test_n = time.time(), 0., 0., 0, 0., 0., 0
    
    model.train()
    for X, y in train_iter:
        print("X: ", X)
        print("y: ", y)
        print("X shape:", X.shape)
        print("y shape:", y.shape)
        print("y dtype:", y.dtype)
        trainer2.zero_grad()
        y_hat = model.forward(X)
        print("y_hat: ", y_hat)
        print("y_hat shape: ", y_hat.shape)
        l = loss(y_hat, y)
        l.backward()
        trainer2.step()
        train_loss += l.item()
        train_acc += (y_hat.argmax(dim=1) == y).sum().item()
        train_n += len(X)
        
    model.eval()
    for X, y in test:
        y_hat = model(X)
        l = loss(y_hat, y)
        test_loss += l.item()
        test_acc += (y_hat.argmax(dim=1) == y).sum().item()
        test_n += len(X)
        
    print("""epoch {}, taked: {:.3f}
    train -> loss: {:.3f}, acc: {:.3f}
    test -> loss: {:.3f}, acc: {:.3f}
    """.format(epoch, time.time() - start, train_loss / train_n, train_acc / train_n,
              test_loss / test_n, test_acc / test_n))
        

In [ ]:
y